Setup date range and weekdays

In [75]:
import random
from datetime import datetime, timedelta, date
import pandas as pd

#set start dates for term
start_date = '2023-01-12'
end_date =  '2023-07-01'

#create date dataframe and add weekdays
s = pd.date_range(start_date, end_date, freq='D').to_series()
d = s.dt.day_name()

df = pd.DataFrame(data=d)
df = df.reset_index(level=0)
df.index = df.index + 1
df.rename(columns={'index':'date', 0: 'weekday'}, inplace = True)


,date,weekday
1,2023-01-12,Thursday
2,2023-01-13,Friday
3,2023-01-14,Saturday
4,2023-01-15,Sunday
5,2023-01-16,Monday


Select Tuesdays and Thursdays

In [76]:
#select specific weekday if desired & add dates to exclude
desired_weekday = ['Tuesday', 'Thursday']
korken_closed = ['2023-03-14', '2023-03-15', '2023-06-29', '2023-06-30']

tt_df = df[df.weekday.isin(desired_weekday) & ~df.date.isin(korken_closed)]

In [77]:
#some stats
index = tt_df.index
number_of_rows = len(index)
print(f"The number of kitchen & cleaning days between {start_date} & {end_date} is {number_of_rows}")


The number of kitchen days between 2023-01-12 & 2023-07-01 is 47


In [78]:
#rename df for function to come
dates_data = tt_df.drop('weekday', axis=1, inplace=False)

,date
1,2023-01-12
6,2023-01-17
8,2023-01-19
13,2023-01-24
15,2023-01-26
20,2023-01-31
22,2023-02-02
27,2023-02-07
29,2023-02-09
34,2023-02-14


In [79]:
import numpy as np

# Sample data for debugging
people_data = {
    'person': [f'Person_{i}' for i in range(15)]
}

people = pd.DataFrame(people_data)

#stats
#print(len(people))
#print(len(dates_data))
#print(len(dates_data) // len(people))

15
47
3


In [122]:
#function to evenly distribute each name to as many dates as needed

def assign_people_to_dates(dates_df, people_df):
    num_dates = len(dates_df)
    num_people = len(people_df)
    
    # Calculate the number of people needed for each date (set to 1 in this case)
    people_per_date = 1
    total_people_needed = people_per_date * num_dates
    
    # Repeat the people list to match the total number of assignments
    people_list = people_df['person'].tolist() * (total_people_needed // num_people)
    remaining_people = total_people_needed % num_people
    if remaining_people > 0:
        people_list += people_df['person'].head(remaining_people).tolist()

    # Create DataFrames for morning and evening assignments
    dates_df['person_kitchen'] = np.array_split(people_list[:total_people_needed], num_dates)
    dates_df['person_cleaning'] = np.array_split(people_list[:total_people_needed], num_dates)

    # Concatenate morning and evening assignments
    kitchen_df = dates_df[['date', 'person_kitchen']].explode('person_kitchen')
    cleaning_df = dates_df[['date', 'person_cleaning']].explode('person_cleaning')

      # Shuffle the 'Name' columns
    kitchen_shuffle = np.random.permutation(kitchen_df['person_kitchen'])
    kitchen_df['person_kitchen'] = kitchen_shuffle
    cleaning_shuffle = np.random.permutation(cleaning_df['person_cleaning'])
    cleaning_df['person_cleaning'] = cleaning_shuffle

    # Combine morning and evening DataFrames and drop the intermediate columns
    result_df = kitchen_df.merge(cleaning_df, on = 'date', how = 'inner')

    return result_df

# Call the function with your DataFrames
assigned_df = assign_people_to_dates(dates_data, people)

assigned_df

,date,person_kitchen,person_cleaning
0,2023-01-12,Person_7,Person_0
1,2023-01-17,Person_2,Person_1
2,2023-01-19,Person_12,Person_12
3,2023-01-24,Person_13,Person_2
4,2023-01-26,Person_9,Person_9
5,2023-01-31,Person_1,Person_7
6,2023-02-02,Person_0,Person_11
7,2023-02-07,Person_12,Person_4
8,2023-02-09,Person_0,Person_10
9,2023-02-14,Person_14,Person_7


In [123]:
#verify counts
result = assigned_df.groupby('person_kitchen').size().reset_index(name='count')
result2 = assigned_df.groupby('person_cleaning').size().reset_index(name='count')

print(result)
print(result2)

   person_kitchen  count
0        Person_0      4
1        Person_1      4
2       Person_10      3
3       Person_11      3
4       Person_12      3
5       Person_13      3
6       Person_14      3
7        Person_2      3
8        Person_3      3
9        Person_4      3
10       Person_5      3
11       Person_6      3
12       Person_7      3
13       Person_8      3
14       Person_9      3
   person_cleaning  count
0         Person_0      4
1         Person_1      4
2        Person_10      3
3        Person_11      3
4        Person_12      3
5        Person_13      3
6        Person_14      3
7         Person_2      3
8         Person_3      3
9         Person_4      3
10        Person_5      3
11        Person_6      3
12        Person_7      3
13        Person_8      3
14        Person_9      3


In [124]:
#check how it looks in table

assigned_df.to_csv('lottocheck.csv', index=False)
